In [6]:
!poetry add langgraph

Using version ^0.2.60 for langgraph

Updating dependencies
Resolving dependencies... (10.3s)Resolving dependencies... (1.3s)Resolving dependencies... (4.3s)Resolving dependencies... (9.5s)

Package operations: 4 installs, 1 update, 0 removals

  - Updating langchain-core (0.3.15 -> 0.3.28): Pending...
  - Updating langchain-core (0.3.15 -> 0.3.28): Downloading... 0%
  - Updating langchain-core (0.3.15 -> 0.3.28): Downloading... 100%
  - Updating langchain-core (0.3.15 -> 0.3.28): Installing...
  - Updating langchain-core (0.3.15 -> 0.3.28)
  - Installing msgpack (1.1.0): Pending...
  - Installing msgpack (1.1.0): Installing...
  - Installing msgpack (1.1.0)
  - Installing langgraph-checkpoint (2.0.9): Pending...
  - Installing langgraph-sdk (0.1.48): Pending...
  - Installing langgraph-sdk (0.1.48): Downloading... 0%
  - Installing langgraph-sdk (0.1.48): Downloading... 100%
  - Installing langgraph-sdk (0.1.48): Installing...
  - Installing langgraph-sdk (0.1.48): Installing...
  - In

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from typing import List, Dict
load_dotenv()

True

In [2]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [39]:
import requests

@tool
def submit_code(codes, requirements_txt=None):
    """This function submits code and requirements to external API which can accept AWS lambdda codes + the contant of requirements.txt and returns the job_id.
  
    Args:
      codes: AWS lambda codes part
      requrirements_txt: python package list needed in AWS lambda codes
    Returns:
      job_id: Job ID that API returns. the format is {"job_id":3}
    """
    url = "http://localhost:8081/v1/job_submission/"
    headers = {'accept': 'application/json', 'Content-Type': 'application/json'}
    if requirements_txt:
        data = {'codes': codes, 'requirements_txt': requirements_txt}
    else:
        data =  {'codes': codes}
    response = requests.post(url, headers=headers, json=data)
    response.raise_for_status() 
    return response.json()['job_id']

@tool
def get_job_output(job_id):
    """
    this function query job output according to job_id. it may take several minutes to complete.

    Args:
      job_id: Job ID that submit_code returns
    Returns:
      return_string: this function return the API endpoint that we can call AWS lambda codes. empty string if any error happens
    """
    import time
    job_status=-1
    url = "http://localhost:8081/v1/job_info/" + str(job_id)
    print(url)
    job_info={}
    while job_status == -1 or job_status == 0 or job_status == 1 or job_status == 2: # 0: accepting 1: locking 2: executing
        response = requests.get(url)
        response.raise_for_status() 
        job_info = response.json()['job_info']
        job_status = job_info['status']
        endpoint=''
        time.sleep(5)
    if job_status == 3:
        return requests.get(job_info['endpoint']).text
    if job_status == 4:
        return ''

        

In [40]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [41]:
tools = [submit_code, get_job_output]

In [42]:
agent = create_react_agent(llm, tools=tools)
inputs = {"messages":[("user","Please write AWS lambda codes which can calculate the largest prime number less than 100. The output format of this AWS lambda function should look like {\"statusCode\":200,\"body\":PRIME_NUMBER}")]}
print_stream(agent.stream(inputs, stream_mode="values"))

================================ Human Message =================================

Please write AWS lambda codes which can calculate the largest prime number less than 100. The output format of this AWS lambda function should look like {"statusCode":200,"body":PRIME_NUMBER}
================================== Ai Message ==================================
Tool Calls:
  submit_code (call_Sfp3AzZ8htvnJOoJSYZoyobV)
 Call ID: call_Sfp3AzZ8htvnJOoJSYZoyobV
  Args:
    codes: import json

def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

def lambda_handler(event, context):
    largest_prime = None
    for num in range(99, 1, -1):
        if is_prime(num):
            largest_prime = num
            break
    return {
        'statusCode': 200,
        'body': largest_prime
    }
    requirements_txt: None
================================= Tool Message =================================
Nam